In [27]:
# Installing necessary packages
#!pip install sentence_transformers
#!pip install pypdf
!pip install faiss-cpu

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [12]:
pwd

'C:\\Users\\bgi340\\Brintha\\mistral_LLM'

In [48]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("C:/Users/bgi340/Brintha/mistral_LLM")

documents=loader.load()
#print(documents)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'C:\\Users\\bgi340\\Brintha\\mistral_LLM\\us_census.pdf', 'page': 0}, page_content='RESEA RCH ARTICL E\nUnderstanding urbanization: A study of\ncensus and satellite-derived urban classes in\nthe United States, 1990-2010\nDeborah Balk\nID\n1,2\n*, Stefan Leyk\n3\n, Bryan Jones\n1,2\n, Mark R. Montgomery\n4\n, Anastasia Clark\n1\n1 CUNY Institute for Demograph ic Researc h, City University of New York, New York, New York, United\nStates of America, 2 Marxe School of Public and Interna tional Affairs, Baruch College, City University of New\nYork, New York, New York, United States of America, 3 University of Colorad o, Boulder, Colorad o, United\nStates of America, 4 Population Council, New York and Stony Brook University , Stony Brook, New York,\nUnited States of America\n* Deborah .Balk@bar uch.cuny. edu\nAbstract\nMost of future population growth will take place in the world’s cities and towns. Yet, there is\nno well-established, consistent way to measure ei

In [49]:
len(final_documents)

89

In [50]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

In [51]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[ 1.44318035e-02 -4.38729525e-02 -4.01028059e-03  2.51430254e-02
  1.33437626e-02 -7.23351212e-03 -3.27955633e-02  1.10515263e-02
 -1.98575538e-02 -1.62747654e-03  1.32948095e-02 -3.35719623e-02
 -8.51220824e-03 -6.65009860e-03  1.82091892e-02 -9.24554188e-03
 -2.86269207e-02  7.42408410e-02  1.31107057e-02  4.37869597e-03
 -7.86713418e-03  9.99575853e-03 -5.31446375e-03 -2.84822322e-02
  5.79482019e-02  6.29990995e-02 -1.28249219e-02 -1.62027031e-02
 -4.07649390e-02 -1.69134989e-01  4.54133302e-02 -1.36565138e-02
  4.25170325e-02  2.28414834e-02  1.88813917e-03 -5.24615794e-02
  4.89198118e-02  5.29558174e-02  1.46600641e-02  3.54188867e-02
 -2.52361730e-04 -2.72850841e-02 -3.19574587e-02 -1.62156466e-02
 -8.53730440e-02  3.69226933e-02 -5.31526729e-02  1.34437596e-02
 -8.33109617e-02  6.13054121e-03  1.74908582e-02 -4.26177680e-02
 -1.29347285e-02  6.35210574e-02 -2.75153611e-02  3.81262191e-02
  1.77539308e-02  4.25947867e-02  6.09496981e-02  6.70677796e-03
 -1.45476721e-02 -2.76231

In [52]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [53]:
## Query using Similarity Search
query="HOW HAS THE URBANIZATION CHANGED OVER THE TIME?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

tion, with that percentage projected to grow to 66 percent by 2050 [1]. Yet the meaning of
urban in such often-cited figures is decidedly unclear: there exists substantial variation across
countries in the urban definitions adopted by their statistical authorities, and countries com-
monly change definitions over time [2]. Even in the United States, where urban definitions
have been well documented and accompanied by census data available in fine spatial detail,
the evolution of concepts and measures over the past few decades has made it difficult to craft
a consistent analytic account of urbanization [3,4]. In other countries lacking comparable doc-
umentation and data, and certainly when comparisons are made across countries, the barriers
to understanding urban change can be formidable (see, for example, continental [5] and global
[6] efforts to harmonize urban delineations).
PLOS ONE | https://doi.or g/10.137 1/journal.po ne.02084 87 December 26, 2018 1 / 20
a1111111111
a1111111111


In [54]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020EE78E2C60> search_kwargs={'k': 3}


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="your_huggingface_api_token"

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [65]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the highest populated city of India?"
hf.invoke(query)

'What is the highest populated city of India?\n\nNew Delhi is the highest populated city of India. It is the capital of India and is located in the northern part of the country. It is the largest commercial center of northern India.\n\nWhat is the highest populated city of India?\n\nNew Delhi is the highest populated city of India. It is the capital of India and is located in the northern part of the country. It is the largest commercial center of northern India.\n\nWhat is the highest populated'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [58]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [59]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [60]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [61]:
query="""DIFFERENCES IN THE
URBAN POPULATION RATE BY STATE
IN 2022"""

In [62]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

the urban population. For the US case, we have shown that the adoption of a 50 percent built-
up threshold (and a simple proportional allocation rule), fails to identify some 30 percent of
the official urban population. Lower built-up thresholds capture more of the official urban
population, and therefore merit careful consideration, as do richer specifications of the built-
up surface that take contiguity and proximity into account. Combined census and satellite
data can be further analyzed for even more detailed and nuanced characterizations of urban
form and systematic evaluation of urban development patterns.
GHSL also holds promise for predictions of future urbanization and urban spatial patterns.
This is a welcomed and long overdue advance in data and methods in both research and policy
circles that have been dominated by use of simple aspatial trend inter